### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from pandas import DatetimeIndex

%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math

### Carregamento dos dados

In [2]:
dataOriginal = pd.read_csv('new_Global_Superstore2.csv', index_col=0, comment='#') 
df = dataOriginal 

### Tratamento dos dados

In [21]:

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
'''
Links:
https://stackoverflow.com/questions/44552031/sklearnstandardscaler-can-i-inverse-the-standardscaler-for-the-model-output
https://stackoverflow.com/questions/56936780/how-do-i-correctly-use-the-inverse-transform-method-when-using-powertransformer
'''


df = dataOriginal
cols = df.columns
scaler = StandardScaler()
scaler.fit(df[cols])

df = scaler.transform(df[cols])

'''
pt = PowerTransformer()
pt.fit(df)

df = pt.transform(df)
scaler = MinMaxScaler(feature_range = (0,1))
df = scaler.fit_transform(df)
'''
print(df)


[[-2.56909239 -0.86822074 -3.30504661 ... -0.59934891  1.22454555
  -0.31475317]
 [-0.38104867  0.43683774  0.12487042 ... -0.59934891  1.22454555
  -1.41402756]
 [-1.47507053 -0.86822074  0.12487042 ... -0.59934891  1.22454555
  -1.41402756]
 ...
 [ 0.71297319 -0.86822074  0.12487042 ...  3.04980699 -0.81662948
  -0.31475317]
 [ 0.71297319  1.74189622  0.12487042 ... -0.59934891 -0.81662948
   1.33415841]
 [-0.38104867 -0.86822074  0.12487042 ... -0.59934891  1.22454555
   1.33415841]]


### Neurónios

In [16]:
#dataset_MLP = dataset.drop(columns=['Order ID', 'Customer ID', 'Customer Name', 'Product ID', 'Product Name', 'Postal Code'])
dataset_MLP = df

#dataset_MLP = dataset_MLP.drop(['Order Priority', 'Market', "City", "Segment", "Ship Mode", "Ship Date"], axis=1)
#print(dataset_MLP.info())

In [17]:
import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

RANDOM_SEED = 2021

print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.10.0


In [18]:


X_MLP = dataset_MLP.drop('Profit', axis=1)
y_MLP = dataset_MLP[['Profit']]

small_dataset = True
if small_dataset: 
    remove_n = 20000
    drop_indices = np.random.choice(dataset_MLP.index, remove_n, replace=False)
    dataset_MLP_subset = dataset_MLP.drop(drop_indices)
    X_MLP = dataset_MLP_subset.drop('Profit', axis=1)
    y_MLP = dataset_MLP_subset[['Profit']]

 
    
    # Convert data
if "Order Date" in X_MLP:
    consider_year = True
    consider_month = True
    consider_day = True
else: 
    consider_year = False
    consider_month = False
    consider_day = False





AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [ ]:

# Onde fui buscar algumas coisas:
# https://www.projectpro.io/recipes/find-optimal-parameters-using-gridsearchcv


X_train, X_test, y_train, y_test = train_test_split(X_MLP, y_MLP, test_size=0.2, random_state=RANDOM_SEED)

number_columns = len(X_scaled.columns)#.count()
#print("Número de colunas: ", number_columns)

def build_model(activation='relu', learning_rate=0.01):
#create a sequential model (with three Layers - Last one is the output)
    model = Sequential()
    model.add(Dense(number_columns, input_dim=number_columns, activation=activation))
    model.add(Dense(6, activation=activation))
    model.add(Dense(1, activation='relu'))
  
     #compile the model
    #Define the Loss function, the otimizer and metrics to be used
    model. compile(
        loss = 'mae',
        optimizer = tf.optimizers.Adam(learning_rate),
        metrics = ['mae', 'mse'])
    return model

model = build_model()
model.summary()
''' Este é melhor, mas mais lento
TUNING_DICT = {
    'activation' : ['relu', 'sigmoid'],
    'learning_rate' : [0.01, 0.001]
}
'''
# = {
#    'activation' : ['relu'],
#    'learning_rate' : [0.01]
#}

TUNING_DICT = {'learning_rate': [0.01,0.03, 0.1],                  
                'activation' : ['relu', 'sigmoid'],
                 }

kf = KFold(n_splits=2, shuffle=True, random_state=RANDOM_SEED)

model = KerasRegressor(build_fn=build_model, epochs=15,batch_size=number_columns)
grid_search = GridSearchCV(estimator = model,
                            param_grid = TUNING_DICT,
                            cv = kf,
                            scoring = 'neg_mean_absolute_error',
                            refit ='True',
                            verbose = 1)

grid_search.fit(X_train, y_train, validation_split=0.2)

print(" Results from Grid Search " )
print(grid_search)
#print("\n The best score across ALL searched params:\n",grid_search.best_score_)
#print("\n The best parameters across ALL searched params:\n",grid_search.best_params_)


In [ ]:
# Overfitting analisys

#Our best model (remember we set refit
best_mlp_model = grid_search.best_estimator_

from livelossplot import PlotLossesKerasTF
best_mlp_model.fit(X_train, y_train, epochs=20,
validation_data=(X_test, y_test),
callbacks=[PlotLossesKerasTF()], verbose=1)

### Análise de overfitting

Pelos gráficos, vemos que o medium average error (mae) dá bastante mal, mas o mse dá bem. Como 20 epochs é um valor considerável, consideramos nesta fase melhorar o tratamento de dados, e depois verificar se há melhorias. 
Isto porque, o tratamento de dados atual para este modelo é quase com o objetivo de o modelo consiguir analisar o dataset.

In [ ]:
 #0btain predictions

predictions = best_mlp_model.predict(X_test)


predictions = predictions.reshape(predictions.shape[0], 1)
predicions_unscaled = scaler_y.inverse_transform(predictions)
predicions_unscaled[:5]

In [ ]:
y_test_unscaled = scaler_y.inverse_transform(y_test)

#Visualising the actual and predicted result
def real_predicted_viz(limit):
    plt.figure(figsize=(14,6))
    plt.plot(y_test_unscaled[:limit], color = 'green', label = 'Actual')
    plt.plot (predicions_unscaled[:limit], color = 'red', label = 'Predicted')
    plt.grid(alpha = 0.3)
    plt.xlabel('Sample')
    plt.ylabel ('Profit')
    plt.title('Real vs Predicted')
    plt.legend()
    plt.show()

#let's Limit to 200 comparisions for better visualization
real_predicted_viz(400)


Pela análise do gráfico, podemos ver que há algumas previsões aproximadamente corretas, mas existem algumas, bem visíveis, que mostram quão errado o nosso modelo está.
Como dito anteriormente, falta tratamento de dados. 

In [ ]:

print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Média: %f Desvio padrão: (%f) com: %r" % (mean, stdev, param))


In [ ]:
#tf.keras.losses.mean_absolute_error(y_test_unscaled, predicions_unscaled)
mae = tf.keras.losses.MeanAbsoluteError()
print("MAE: ", mae(y_test_unscaled, predicions_unscaled).numpy())
#mse = tf.keras.losses.mean_squared_error(y_test_unscaled, predicions_unscaled).numpy()
mse = np.mean(np.square(y_test_unscaled - predicions_unscaled))
print("MSE: ", mse)
print("RMSE: ", math.sqrt(mse))